Pipeline = Tokenizer -> Model -> Post-processing

Data : Text -> Input IDs -> Logits -> Predictions

#Tokenization

**Steps :**
Raw text -> Tokens -> Special tokens (added so that the model locate the begining and end of each sentence) -> PAdding (if a sentence by-passes the max length, it is truncked and marked with [PAD]) -> Input ID

In [1]:
# AutoTokenizer API
!pip install transformers

from transformers import AutoTokenizer

checkpoint="bert-base-uncased";
tokenizer = AutoTokenizer.from_pretrained(checkpoint);

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
raw_inputs = ['My name is Zeineb and i am passionate about tennis.', 'Rafael Nadal is my favourite tennis player.']
inputs = tokenizer(raw_inputs, padding = True, truncation =True, return_tensors="pt");
inputs

{'input_ids': tensor([[  101,  2026,  2171,  2003, 27838,  3170,  2497,  1998,  1045,  2572,
         13459,  2055,  5093,  1012,   102],
        [  101, 10999, 23233,  2389,  2003,  2026,  8837,  5093,  2447,  1012,
           102,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

In [4]:
inputs.input_ids

tensor([[  101,  2026,  2171,  2003, 27838,  3170,  2497,  1998,  1045,  2572,
         13459,  2055,  5093,  1012,   102],
        [  101, 10999, 23233,  2389,  2003,  2026,  8837,  5093,  2447,  1012,
           102,     0,     0,     0,     0]])

In [6]:
inputs.attention_mask #indicated with 0 the ids that the model doesn't have to pay attention to

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])

#Model

Il s'agit d'instancier un modèle

In [7]:
#Use AutoModel APIs to pass the token input to the model
#Here we used an AutoModelAPI adapted for classification tasks

from transformers import AutoModelForSequenceClassification

checkpoint="bert-base-uncased";
model = AutoModelForSequenceClassification.from_pretrained(checkpoint);
output = model(**inputs)
print(output);

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassifierOutput(loss=None, logits=tensor([[ 0.4580, -0.0160],
        [ 0.3933, -0.0441]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [8]:
output.logits

tensor([[ 0.4580, -0.0160],
        [ 0.3933, -0.0441]], grad_fn=<AddmmBackward0>)

#Postprocessing

In [10]:
#Applying Softmax layers
import torch

predictions = torch.nn.functional.softmax(output.logits, dim=-1)
print(predictions)

tensor([[0.6163, 0.3837],
        [0.6076, 0.3924]], grad_fn=<SoftmaxBackward0>)


In [11]:
model.config.id2label # Get the labels of the model

{0: 'LABEL_0', 1: 'LABEL_1'}